### Maps,Hash Tables and Skip Lists
#### 定义:
It represents an abstraction known as a dictionary in which unique keys are mapped to associated values.

#### 抽象数据结构
base
* M[k] 查找
* M[k]=v 增加
* del M[k] 删除
* len(M)   返回items数量
* iter(M)  迭代

addition
* k in M    包含
* M.get(k,d=None) 默认get
* M.setdefault(K,d) 默认set
* M.pop(k,d=None) 默认去除
* M.popitem() 去除并返回键值对
* M.keys()
* M.values()
* M.items()
* M.update(M2) 字典合并
* M=M2
* M!=M2
##### 我们将要实现的字典类型
![Screenshot from 2017-04-05 09-58-40.png](https://ooo.0o0.ooo/2017/04/05/58e44f6888401.png)

In [2]:
#我们的基础字典,借用python的字典抽象类
from collections.abc import MutableMapping
class MapBase(MutableMapping):
    class _Item:
        __slot__="_key","_value"
        def __init__(self,k,v):
            self._key=k
            self._value=v
            
        def __eq__(self,other):
            return self._key==other._key
        
        def __ne__(self,other):
            return not (self==other)
        
        def __lt__(self,other):
            return self._key<other._key

##### 无序字典

In [7]:
class UnsortedTableMap(MapBase):
    def __init__(self):
        self._table=[]
        
    def __getitem__(self,k):
        for item in self._table:
            if k==item._key:
                return item._value
        raise KeyError("Key Error:{}".format(k))
        
    def __setitem__(self,k,v):
        for item in self._table:
            if k==item._key:
                item._value=v
                return
        self._table.append(self._Item(k,v))
    def __delitem__(self,k):
        for j in range(len(self._table)):
            if k==self._table[j]._key:
                self._table.pop(j)
                return 
        raise KeyError("Key Error:{}".format(k))
        
    def __len__(self):
        return len(self._table)
    
    def __iter__(self):
        for item in self._table:
            yield item._key
            
a=UnsortedTableMap()
a["a"]=1
print(a["a"])
a["a"]=2
print(a["a"])
del a["a"]
print(a["a"])

1
2


KeyError: 'Key Error:a'

#### hash table(散列表)
使用一个固定长度为n的查找表格,将key k存储在index 为k的表格中,基本的字典操作getitem,setitem,delitem 的时间复杂度就变为$O(1)$
达到这样的目的有两种困难:
1. 查找表格的长度不能远大于item的个数
2. 必须找到一个hash函数,将key映射到查找表格上,并减少hash冲突(多个键同一个index)
对于hash冲突的key,使用hash桶
![Screenshot from 2017-04-05 10-31-31.png](https://ooo.0o0.ooo/2017/04/05/58e4571585a7c.png)

##### hash函数
hash函数将任意键转换为查找表范围内的一个整数
hash函数可以分为两部分:
1. a hash code that maps a key k to an integer(hash code生产)
2. a compression function that maps the hash code to an integer within a range of indices(压缩,取模)
![Screenshot from 2017-04-05 10-55-35.png](https://ooo.0o0.ooo/2017/04/05/58e45cb8527fd.png)

##### hash code
1. 用bit表示作为整数  
任何数据类型在计算机内都是位表示 64位的float可以转换位32位的integer--直接截取32位高位,或者将32位高位与32位低位通过各种方式混合.
2. hash code多项式
考虑位置,将每个元素的hash值结合
![Screenshot from 2017-04-05 11-14-06.png](https://ooo.0o0.ooo/2017/04/05/58e462310146a.png)

##### hash code in python
the immutable int, float, str, tuple, and frozenset classes produce robust hash codes.使用python的不可变类型来构造hash code  
使用元组
```
def hash (self):
    return hash( (self. red, self. green, self. blue) )
```
只要定义\__eq\__,必须定义\__hash\__

##### 压缩方法
1. The Division Method  
$i\,mod\,N$  
风险是N是非质数的话,非常容易产生hash冲突
2. The MAD Method (Multiply-Add-and-Divide)  
${[}(ai+b)\,mod\,p{]}\,mod\,N$  
 * N是查找表格的大小 
 * p是比N大的质数
 * a,b是[0,p-1]的随机数,a>0



#### 应对hash冲突的两种方法
##### Separate Chaining(分别链接)
原本Hash表每个元素是一个输入数据的数据结构类型，现在把每个元素改成一个由该数据结构类型构成的指针链表。这样，当发生冲突时，只要在该指针链表的尾端或首端插入该值即可。
![Screenshot from 2017-04-05 12-56-02.png](https://ooo.0o0.ooo/2017/04/05/58e478ff57099.png)
哈希桶的size n/查找表大小 N称为load factor 负载因子,越小越好

##### 开放地址法
线性插入法
当a[j]被占据,我们选择a[(j+1) mod n],这样一直下去.
![Screenshot from 2017-04-05 13-01-56.png](https://ooo.0o0.ooo/2017/04/05/58e47a56b9784.png)
容易发生团聚现象,需要rehash

#### hash表的效率
![Screenshot from 2017-04-05 13-06-56.png](https://ooo.0o0.ooo/2017/04/05/58e47b8759a7c.png)
效率与负债因子相关,期望是n/N,但最差的情况下,hash函数把所有item都分到一个桶中.

#### python hash table实现

In [20]:
from random import randrange
class HashMapBase(MapBase):
    def __init__(self,cap=11,p=109345121):
        self._table=cap*[None]
        self._n=0
        self._prime=p
        self._scale=1+randrange(p-1) #a of mad
        self._shift=randrange(p) #b of mad
        
    def _hash_function(self,k):
        return (hash(k)*self._scale+self._shift)%self._prime%len(self._table)
    
    def __len__(self):
        return self._n
    
    def __getitem__(self,k):
        j=self._hash_function(k)
        return self._bucket_getitem(j,k)
    
    def __setitem__(self,k,v):
        j=self._hash_function(k)
        self._bucket_setitem(j,k,v)
        if self._n>len(self._table)//2:
            self._resize(2*len(self._table)-1) #负载因子永远小于0.5
            
    def __delitem__(self,k):
        j=self._hash_function(k)
        self._bucket_delitem(j,k)
        self._n-=1
        
    def _resize(self,c):
        old=list(self.items())
        self._table=c*[None]
        self._n=0
        for(k,v) in old:
            self[k]=v
            

In [23]:
#分别链接法
class ChainHashMap(HashMapBase):
    def _bucket_getitem(self,j,k):
        bucket=self._table[j]
        if bucket is None:
            raise KeyError("wrong key {}".format(k))
        return bucket[k]
    
    def _bucket_setitem(self,j,k,v):
        if self._table[j] is None:
            self._table[j]=UnsortedTableMap() #以一个无序map作为桶
        oldsize=len(self._table[j])
        self._table[j][k]=v
        if len(self._table[j])>oldsize:
            self._n+=1
            
    def _bucket_delitem(self,j,k):
        bucket=self._table[j]
        if bucket is None:
            raise KeyError("wrong key {}".format(k))
        del bucket[k]
        
    def __iter__(self):
        for bucket in self._table:
            if bucket:
                for key in bucket:
                    yield key

b=ChainHashMap()
b["1"]=2
print(b["1"])

2


#### 排序查找表Sorted Search Tables
在查找表中按key的顺序进行存储
![Screenshot from 2017-04-05 14-42-43.png](https://ooo.0o0.ooo/2017/04/05/58e491f8cd72a.png)
最大的优势是能用二分法进行查找,比如\__contains\__方法可以用二分法重写

In [65]:
class SortedTableMap(MapBase):
    def _find_index(self,k,low,high):
        if high==low:
            return high
        else:
            mid=(high+low)//2
            if k==self._table[mid]._key:
                return mid
            elif k<self._table[mid]._key:
                return self._find_index(k,low,mid)
            else:
                return self._find_index(k,mid+1,high)
            
    def __init__(self):
        self._table=[]
        
    def __len__(self):
        return len(self._table)
    
    def __getitem__(self,k):
        j=self._find_index(k,0,len(self._table))
        if  j==len(self._table) or self._table[j]._key!=k:
            raise KeyError("wrong key {}".format(k))
        return self._table[j]._value
    def __setitem__(self,k,v):
        j=self._find_index(k,0,len(self._table))
        if j<len(self._table) and self._table[j]._key==k:
            self._table[j]._value=v
        else:
            self._table.insert(j,self._Item(k,v))
            
    def __delitem__(self,k):
        j=self._find_index(k,0,k,0,len(self._table) if len(self._table)>0 else 1)
        if j==len(self._table) or self._table[j]._key!=k:
            raise KeyError("wrong key {}".format(k))
        self._table.pop(j)
        
    def __iter__(self):
        for item in self._table:
            yield item._key
            
    def __contains__(self,k):
        if len(self._table)==0:
            return False
        else:
            j=self._find_index(k,0,len(self._table)) 
            if j==len(self._table) or self._table[j]._key!=k:
                return False
            return True
        
            
b=SortedTableMap()
b[1]="a"
print(list(b))
print(b[1])
b[2]=2
print(list(b))
b[3]=3
b[3]=5
b[0]=0
print(list(b))
1 in b

[1]
a
[1, 2]
[0, 1, 2, 3]


True

##### 排序查找表的效率
![Screenshot from 2017-04-05 15-52-24.png](https://ooo.0o0.ooo/2017/04/05/58e4a2533eb9e.png)

#### 集合,包,多重字典

##### 集合
集合是不重复的无序元素的组合,集合跟map非常相似,区别只在于查找表不存item,直接存储值.
##### 包
python中的counter,它对于一个list,返回以每个独特元素为key,每个特殊元素出现的次数为value的字典.
##### 多重字典
a common implementation approach is to use a standard map in which the value associated with a key is itself a container class storing any number of associated values.
就是一个value是list的字典


In [3]:
class MultiMap:
    def __init__(self):
        self._map=dict()
        self._n=0
        
    def add(self,k,v):
        container=self._map.setdefault(k,[])
        container.append(v)
        self._n+=1
        
    def pop(self,k):
        container=self._map[k]
        v=container.pop()
        #if container is emptylist del k
        if len(container)==0:
            del self._map[k]
        self._n-=1
        return (k,v)
    
    def find(self,k):
        return self._map[k]
    
    def __iter__(self):
        for key,value in self._map.items():
                yield key,value
                
c=MultiMap()
c.add(1,2)
c.add(3,4)
print(list(c))
                
    
        

[(1, [2]), (3, [4])]
